In [2]:
import pandas as pd
import numpy as np

def clean_text(df):

    df.columns = df.columns.str.lower()
    str_columns = df.select_dtypes(include='object').columns
    
    for col in str_columns:
        df[col] = df[col].str.lower().str.replace(' ', '_')
    
    return df

def replace_dates_with_nan(value):
    if isinstance(value, str):
        cleaned_value = value.replace('_', ' ').replace(',', '')
        try:
            parsed_date = pd.to_datetime(cleaned_value, format='%B %d %Y', errors='raise')
            return np.nan  
        except ValueError:
            return value
    return value

def convert_float(number, characters):
    try:
        for char in characters:
            number = number.replace(char, '')
        return float(number)
    except ValueError:
        return float('nan')

def convert_column_to_float(df, column_name, characters):
    df[column_name] = df[column_name].apply(lambda x: convert_float(x, characters))
    return df

def isFloat(str):
    try:
        float(str)
        return True
    except ValueError:
        return False
    
def remove_digit(string):
    if string.isdigit() or isFloat(string):
        string=''
    return string

def normalize_and_encode_ratings(df,column_name):    
    rating_labels = {
        'unrated': 0,
        'everyone': 1,
        'everyone_10+': 2,
        'teen': 3,
        'mature_17+': 4,
        'adults_only_18+': 5
    }
    df['rating_label'] = df[column_name].map(rating_labels).fillna(-1)
    df['rating_label']=df['rating_label'].astype('int')
    return df

def clean_date(date_str):
    try:
        cleaned_date = date_str.replace('_', ' ').replace(',', '')
        return pd.to_datetime(cleaned_date, format='%B %d %Y', errors='coerce')
    except Exception as e:
        print(f"Error processing date: {date_str} - {e}")
        return pd.NaT 
    
def replace_dates_with_null(df, column, months_list):
    months_list = [month.lower() for month in months_list]

    def contains_month(value):
        if isinstance(value, str):  
            value_lower = value.lower()
            for month in months_list:
                if month in value_lower:
                        return np.nan 
        return value  
                            
    df[column] = df[column].apply(contains_month)
                            
    return df

def map_type_in_place(df, type_col):
    type_mapping = {'free': 0, 'paid': 1}
    df[type_col] = df[type_col].map(type_mapping)

def remove_duplicates(df):
    df= df.drop_duplicates()
    return df

def create_app_id_column(df, app_name_col, new_id_col):
    unique_app_names = df[app_name_col].unique()
    app_id_mapping = {app_name: idx + 1 for idx, app_name in enumerate(unique_app_names)}
    df[new_id_col] = df[app_name_col].map(app_id_mapping)
    cols = [new_id_col] + [col for col in df.columns if col not in [new_id_col]]
    df = df[cols]
    return df



def transform(df_playstore):
    df_playstore=remove_duplicates(df_playstore)
    df_playstore=clean_text(df_playstore)
    df_playstore = convert_column_to_float(df_playstore, 'price', ['$'])
    df_playstore=convert_column_to_float(df_playstore,'size',['m'])
    df_playstore=convert_column_to_float(df_playstore,'installs',[',','+'])
    df_playstore=convert_column_to_float(df_playstore,'reviews',[])
    df_playstore['category'] = df_playstore['category'].apply(remove_digit)
    df_playstore['app']=df_playstore['app'].apply(remove_digit)
    df_playstore['genres']=df_playstore['genres'].apply(remove_digit)
    df_playstore=normalize_and_encode_ratings(df_playstore,'content rating')
    df_playstore['last updated'] = df_playstore['last updated'].apply(clean_date)
    df_playstore=replace_dates_with_null(df_playstore,'genres',months_list = ["january", "february", "march", "april", "may", "june", "july", "august", "september", "october", "november", "december"])
    map_type_in_place(df_playstore, 'type')
    df_playstore.drop(df_playstore[df_playstore['type'] == '0'].index, inplace=True)
    df_playstore=create_app_id_column(df_playstore,'app','app_id')
    return df_playstore


In [3]:
df_playstore=pd.read_csv('csv/googleplaystore.csv')
df = transform(df_playstore)

/var/folders/d2/zqv2mpg17tzb0413h3b_jwb00000gn/T/ipykernel_71399/3044256052.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].str.lower().str.replace(' ', '_')
/var/folders/d2/zqv2mpg17tzb0413h3b_jwb00000gn/T/ipykernel_71399/3044256052.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].apply(lambda x: convert_float(x, characters))
/var/folders/d2/zqv2mpg17tzb0413h3b_jwb00000gn/T/ipykernel_71399/3044256052.py:109: SettingWithCopyWarning: 
A value is tr

In [4]:
df = df.drop_duplicates(subset='app_id', keep='first')


In [5]:
df

,app_id,app,category,rating,reviews,size,installs,type,price,content rating,genres,last updated,current ver,android ver,rating_label
0,1,photo_editor_&_candy_camera_&_grid_&_scrapbook,art_and_design,4.1,159.0,19.0,10000.0,0.0,0.0,everyone,art_&_design,2018-01-07,1.0.0,4.0.3_and_up,1
1,2,coloring_book_moana,art_and_design,3.9,967.0,14.0,500000.0,0.0,0.0,everyone,art_&_design;pretend_play,2018-01-15,2.0.0,4.0.3_and_up,1
2,3,"u_launcher_lite_–_free_live_cool_themes,_hide_...",art_and_design,4.7,87510.0,8.7,5000000.0,0.0,0.0,everyone,art_&_design,2018-08-01,1.2.4,4.0.3_and_up,1
3,4,sketch_-_draw_&_paint,art_and_design,4.5,215644.0,25.0,50000000.0,0.0,0.0,teen,art_&_design,2018-06-08,varies_with_device,4.2_and_up,3
4,5,pixel_draw_-_number_art_coloring_book,art_and_design,4.3,967.0,2.8,100000.0,0.0,0.0,everyone,art_&_design;creativity,2018-06-20,1.1,4.4_and_up,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10836,9635,sya9a_maroc_-_fr,family,4.5,38.0,53.0,5000.0,0.0,0.0,everyone,education,2017-07-25,1.48,4.1_and_up,1
10837,9636,fr._mike_schmitz_audio_teachings,family,5.0,4.0,3.6,100.0,0.0,0.0,everyone,education,2018-07-06,1.0,4.1_and_up,1
10838,9637,parkinson_exercices_fr,medical,NaN,3.0,9.5,1000.0,0.0,0.0,everyone,medical,2017-01-20,1.0,2.2_and_up,1
10839,9638,the_scp_foundation_db_fr_nn5n,books_and_reference,4.5,114.0,NaN,1000.0,0.0,0.0,mature_17+,books_&_reference,2015-01-19,varies_with_device,varies_with_device,4


In [6]:
df.loc[10838]

app_id                              9637
app               parkinson_exercices_fr
category                         medical
rating                               NaN
reviews                              3.0
size                                 9.5
installs                          1000.0
type                                 0.0
price                                0.0
content rating                  everyone
genres                           medical
last updated         2017-01-20 00:00:00
current ver                          1.0
android ver                   2.2_and_up
rating_label                           1
Name: 10838, dtype: object

In [7]:
def transform_review(df):
    
    def clean_text(df):
        df.columns = df.columns.str.lower()
        str_columns = df.select_dtypes(include='object').columns
        
        for col in str_columns:
            df[col] = df[col].str.lower().str.replace(' ', '_')
        
        return df
    
    def drop_na_reviews(df):
        df.dropna(subset=['translated_review'], inplace=True)
        return df

    def create_app_id_column(df, app_name_col, new_id_col):
        unique_app_names = df[app_name_col].unique()
        app_id_mapping = {app_name: idx + 1 for idx, app_name in enumerate(unique_app_names)}
        df[new_id_col] = df[app_name_col].map(app_id_mapping)
        cols = [new_id_col] + [col for col in df.columns if col not in [new_id_col]]
        df = df[cols]
        return df

    def create_reviews_summary(df, app_name_col, app_id_col, sentiment_col, subjectivity_col):
        summary = df.groupby([app_id_col, app_name_col]).agg(
            number_of_reviews=(app_id_col, 'count'),
            average_sentiment_analysis=(sentiment_col, 'mean'),
            average_sentiment_subjectivity=(subjectivity_col, 'mean')
        ).reset_index()
        
        df = pd.merge(df, summary, on=[app_id_col, app_name_col], how='left')
        
        def map_sentiment(polarity):
            if polarity > 0:
                return 'positive'
            elif polarity < 0:
                return 'negative'
            else:
                return 'neutral'
            
        def map_subjectivity(subjectivity):
            if subjectivity > 0.5:
                return 'fact'
            elif subjectivity < 0.5:
                return 'opinion'
            elif subjectivity == 0.5:
                return 'mixed'
            
        df['sentiment_category'] = df['average_sentiment_analysis'].apply(map_sentiment)
        df['subjectivity_category'] = df['average_sentiment_subjectivity'].apply(map_subjectivity)
        
        return df

    # Process the DataFrame
    df = clean_text(df)
    df = drop_na_reviews(df)
    df = create_app_id_column(df, 'app', 'app_id')
    df = create_reviews_summary(df, 'app', 'app_id', 'sentiment_polarity', 'sentiment_subjectivity')
    
    return df


In [8]:
review_df = pd.read_csv('csv/googleplaystore_user_reviews.csv')

In [9]:
cleaned_review = transform_review(review_df)

columns_to_remove = [
    'translated_review', 'sentiment_polarity', 'sentiment_subjectivity',
    'sentiment_category', 'subjectivity_category'
]

cleaned_review = cleaned_review.drop(columns=columns_to_remove, errors='ignore')
cleaned_review = cleaned_review.drop_duplicates(subset='app_id', keep='first')


In [10]:
cleaned_review

,app_id,app,sentiment,number_of_reviews,average_sentiment_analysis,average_sentiment_subjectivity
0,1,10_best_foods_for_you,positive,194,0.470733,0.495455
194,2,104_找工作_-_找工作_找打工_找兼職_履歷健檢_履歷診療室,positive,40,0.392405,0.545516
234,3,11st,negative,39,0.185943,0.455340
273,4,1800_contacts_-_lens_store,positive,80,0.318145,0.591098
353,5,1line_–_one_line_with_one_touch,negative,38,0.196290,0.557315
...,...,...,...,...,...,...
37239,861,hotels.com:_book_hotel_rooms_&_find_vacation_d...,positive,68,0.101622,0.545444
37307,862,hotspot_shield_free_vpn_proxy_&_wi-fi_security,negative,34,0.251765,0.393284
37341,863,hotstar,neutral,32,0.038178,0.493964
37373,864,hotwire_hotel_&_car_rental_app,negative,33,0.187029,0.459717


In [11]:
merged_df = df.merge(cleaned_review, on='app', how='left')
merged_df.drop(columns='number_of_reviews', errors='coerce' )

,app_id_x,app,category,rating,reviews,size,installs,type,price,content rating,genres,last updated,current ver,android ver,rating_label,app_id_y,sentiment,average_sentiment_analysis,average_sentiment_subjectivity
0,1,photo_editor_&_candy_camera_&_grid_&_scrapbook,art_and_design,4.1,159.0,19.0,10000.0,0.0,0.0,everyone,art_&_design,2018-01-07,1.0.0,4.0.3_and_up,1,NaN,NaN,NaN,NaN
1,2,coloring_book_moana,art_and_design,3.9,967.0,14.0,500000.0,0.0,0.0,everyone,art_&_design;pretend_play,2018-01-15,2.0.0,4.0.3_and_up,1,385.0,negative,0.152652,0.64154
2,3,"u_launcher_lite_–_free_live_cool_themes,_hide_...",art_and_design,4.7,87510.0,8.7,5000000.0,0.0,0.0,everyone,art_&_design,2018-08-01,1.2.4,4.0.3_and_up,1,NaN,NaN,NaN,NaN
3,4,sketch_-_draw_&_paint,art_and_design,4.5,215644.0,25.0,50000000.0,0.0,0.0,teen,art_&_design,2018-06-08,varies_with_device,4.2_and_up,3,NaN,NaN,NaN,NaN
4,5,pixel_draw_-_number_art_coloring_book,art_and_design,4.3,967.0,2.8,100000.0,0.0,0.0,everyone,art_&_design;creativity,2018-06-20,1.1,4.4_and_up,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9634,9635,sya9a_maroc_-_fr,family,4.5,38.0,53.0,5000.0,0.0,0.0,everyone,education,2017-07-25,1.48,4.1_and_up,1,NaN,NaN,NaN,NaN
9635,9636,fr._mike_schmitz_audio_teachings,family,5.0,4.0,3.6,100.0,0.0,0.0,everyone,education,2018-07-06,1.0,4.1_and_up,1,NaN,NaN,NaN,NaN
9636,9637,parkinson_exercices_fr,medical,NaN,3.0,9.5,1000.0,0.0,0.0,everyone,medical,2017-01-20,1.0,2.2_and_up,1,NaN,NaN,NaN,NaN
9637,9638,the_scp_foundation_db_fr_nn5n,books_and_reference,4.5,114.0,NaN,1000.0,0.0,0.0,mature_17+,books_&_reference,2015-01-19,varies_with_device,varies_with_device,4,NaN,NaN,NaN,NaN


In [12]:
merged_df.columns

Index(['app_id_x', 'app', 'category', 'rating', 'reviews', 'size', 'installs',
       'type', 'price', 'content rating', 'genres', 'last updated',
       'current ver', 'android ver', 'rating_label', 'app_id_y', 'sentiment',
       'number_of_reviews', 'average_sentiment_analysis',
       'average_sentiment_subjectivity'],
      dtype='object')

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Load your data
data = merged_df.copy()

# Define success based on downloads
data['success'] = (data['installs'] > 100000).astype(int)

# Define features and target variable
features = ['category', 'size', 'type', 'price', 'content rating', 'genres', 'current ver', 'android ver', 'sentiment']
target = 'success'

# Prepare data
X = data[features]
y = data[target]

# Identify missing values
print(X.isna().sum())

# Handle missing values
# For numerical features, you can use SimpleImputer to fill missing values with the median
numeric_features = ['size', 'price']
categorical_features = ['category', 'type', 'content rating', 'genres', 'current ver', 'android ver', 'sentiment']

# Create a column transformer with imputers
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),  # Impute missing values for numeric features
            ('scaler', StandardScaler())  # Normalize numeric features
        ]), numeric_features),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),  # Impute missing values for categorical features
            ('encoder', OneHotEncoder(handle_unknown='ignore'))  # Encode categorical features
        ]), categorical_features)
    ])

# Create a pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier())
])

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))


category             0
size              1541
type                 2
price                1
content rating       1
genres               1
current ver          8
android ver          3
sentiment         8823
dtype: int64
              precision    recall  f1-score   support

           0       0.78      0.86      0.82      1682
           1       0.77      0.66      0.71      1210

    accuracy                           0.78      2892
   macro avg       0.78      0.76      0.77      2892
weighted avg       0.78      0.78      0.77      2892



Interpreting Results
Class 0 (Not Successful):
The model performs better in terms of recall (86%), meaning it identifies a higher proportion of true negatives (non-successful apps). However, precision is slightly lower at 78%, indicating some false positives.
Class 1 (Successful):
The model has lower recall (66%) for successful apps, meaning it identifies fewer true positives. Precision is also slightly lower at 77%, suggesting some false positives in predictions of success.
Accuracy:
An accuracy of 78% is decent, but it might not reflect the model’s performance well if there is a class imbalance (i.e., if one class is much more frequent than the other).
Balanced Performance:
The F1-score for class 0 is higher than for class 1, indicating that the model performs better in identifying non-successful apps. Improving recall for class 1 could be beneficial.